<a href="https://colab.research.google.com/github/present42/PyTorchPractice/blob/main/Fluent_Python_ch12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chapter 12. Special Methods for Sequences

Implementation of `Vector` class with following properties:
 1. Basic sequence protocol: `__len__` and `__getitem__`
 2. Safe representation of instances with many items
 3. Proper slicing support, producing new Vector instances
 4. Aggregate hashing, taking  into account every contained element value
 5. Custom formatting language extension

# Vector Take #1: Vector2d Compatible

best practice for a sequence constructor is to take the data as iterable argument in the constructor.

In [1]:
# vector_v1.py
from array import array
import reprlib
import math

class Vector:
  typecode = 'd'

  def __init__(self, components):
    # self._components - "protected" instance attribute will hold an array with the Vector components
    self._components = array(self.typecode, components)

  def __iter__(self):
    return iter(self._components)

  def __repr__(self):
    components = reprlib.repr(self._components)
    components = components[components.find('['):-1]
    return f'Vector({components})'

  def __str__(self):
    return str(tuple(self))

  def __bytes__(self):
    return (bytes([ord(self.typecode)]) +
            bytes(self._components))

  def __eq__(self, other):
    return tuple(self) == tuple(other)

  def __abs__(self):
    return math.hypot(*self)

  def __bool__(self):
    return bool(abs(self))

  @classmethod
  def frombytes(cls, octets):
    typecode = chr(octets[0])
    memv = memoryview(octets[1:]).cast(typecode)
    return cls(memv)

In [ ]:
tuple(Vector([3.1, 4.2]))

(3.1, 4.2)

In [ ]:
Vector((3, 4, 5))

Vector([3.0, 4.0, 5.0])

In [ ]:
Vector(range(10))

Vector([0.0, 1.0, 2.0, 3.0, 4.0, ...])

## Protocols and Duck Typing

You don't need to inherit from any special class to create a fully functional sequence type in Python; you just need to implement the methods that fulfill the sequence protocol

In [ ]:
# Code from Ex1-1
import collections

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
  ranks = [str(n) for n in range(2, 11)] + list('JQKA')
  suits = 'spades diamonds clubs hearts'.split()

  def __init__(self):
    self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]

  def __len__(self):
    return len(self._cards)

  def __getitem__(self, position):
    return self._cards[position]


The above `FrenchDeck` class is a sequence even if it subclasses `object`. We say it is a sequence because it behaves like one, and that is what matters.

# Vector Take #2: A Sliceable Sequence

In [ ]:
# vector_v2.py
from array import array
import reprlib
import math

class Vector:
  typecode = 'd'

  def __init__(self, components):
    # self._components - "protected" instance attribute will hold an array with the Vector components
    self._components = array(self.typecode, components)

  def __iter__(self):
    return iter(self._components)

  def __repr__(self):
    components = reprlib.repr(self._components)
    components = components[components.find('['):-1]
    return f'Vector({components})'

  def __str__(self):
    return str(tuple(self))

  def __bytes__(self):
    return (bytes([ord(self.typecode)]) +
            bytes(self._components))

  def __eq__(self, other):
    return tuple(self) == tuple(other)

  def __abs__(self):
    return math.hypot(*self)

  def __bool__(self):
    return bool(abs(self))

  def __len__(self):
    return len(self._components)

  def __getitem__(self, index):
    return self._components[index]

  @classmethod
  def frombytes(cls, octets):
    typecode = chr(octets[0])
    memv = memoryview(octets[1:]).cast(typecode)
    return cls(memv)

In [ ]:
v1 = Vector([3, 4, 5])

In [ ]:
len(v1)

3

In [ ]:
v1[0], v1[-1]

(3.0, 5.0)

In [ ]:
v7 = Vector(range(7))
v7[1:4]

array('d', [1.0, 2.0, 3.0])

## how slicing works

In [ ]:
class MySeq:
  def __getitem__(self, index):
    return index

In [ ]:
s = MySeq()

In [ ]:
s[1]

1

In [ ]:
s[1:4]

slice(1, 4, None)

In [ ]:
s[1:4:2]

slice(1, 4, 2)

In [ ]:
s[1:4:2, 9]

(slice(1, 4, 2), 9)

In [ ]:
s[1:4:2, 7:9] # tuple may even hold several slice objects

(slice(1, 4, 2), slice(7, 9, None))

In [ ]:
slice

slice

In [ ]:
dir(slice)

['__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'indices',
 'start',
 'step',
 'stop']

In [ ]:
'ABCDE'[:10:2] == 'ABCDE'[0:5:2]

True

In [ ]:
slice(None, 10, 2).indices(5) # indices exposes the tricky logic that's implemented in the built-in seq
                              # to gracefully handle missing or negative indices

(0, 5, 2)

In [ ]:
slice(-3, None, None).indices(5)

(2, 5, 1)

In our vector code, we'll not need the `slice.indices()` method because when we get a slice argument we'll delegate its handling t the _components array.

In [2]:
# vector_v2.py
from array import array
import operator
import reprlib
import math

class Vector:
  typecode = 'd'

  def __init__(self, components):
    # self._components - "protected" instance attribute will hold an array with the Vector components
    self._components = array(self.typecode, components)

  def __iter__(self):
    return iter(self._components)

  def __repr__(self):
    components = reprlib.repr(self._components)
    components = components[components.find('['):-1]
    return f'Vector({components})'

  def __str__(self):
    return str(tuple(self))

  def __bytes__(self):
    return (bytes([ord(self.typecode)]) +
            bytes(self._components))

  def __eq__(self, other):
    return tuple(self) == tuple(other)

  def __abs__(self):
    return math.hypot(*self)

  def __bool__(self):
    return bool(abs(self))

  def __len__(self):
    return len(self._components)

  def __getitem__(self, key):
    if isinstance(key, slice):
      cls = type(self)
      return cls(self._components[key])
    index = operator.index(key) # operator.index() calls the __index__ special method
    return self._components[index]

  @classmethod
  def frombytes(cls, octets):
    typecode = chr(octets[0])
    memv = memoryview(octets[1:]).cast(typecode)
    return cls(memv)

In [ ]:
v7 = Vector(range(7))

In [ ]:
v7[-1]

6.0

In [ ]:
v7[1:4]

Vector([1.0, 2.0, 3.0])

In [ ]:
v7[-1:]

Vector([6.0])

In [ ]:
v7[1, 2] # Vector does not support multidimensional indexing

TypeError: 'tuple' object cannot be interpreted as an integer

# Vector Take #3: Dynamic Attribute Access

In [11]:
# vector_v3.py
from array import array
import operator
import reprlib
import math

class Vector:
  typecode = 'd'
  __match_args__ = ('x', 'y', 'z', 't')

  def __init__(self, components):
    # self._components - "protected" instance attribute will hold an array with the Vector components
    self._components = array(self.typecode, components)

  def __iter__(self):
    return iter(self._components)

  def __repr__(self):
    components = reprlib.repr(self._components)
    components = components[components.find('['):-1]
    return f'Vector({components})'

  def __str__(self):
    return str(tuple(self))

  def __bytes__(self):
    return (bytes([ord(self.typecode)]) +
            bytes(self._components))

  def __eq__(self, other):
    return tuple(self) == tuple(other)

  def __abs__(self):
    return math.hypot(*self)

  def __bool__(self):
    return bool(abs(self))

  def __len__(self):
    return len(self._components)

  def __getitem__(self, key):
    if isinstance(key, slice):
      cls = type(self)
      return cls(self._components[key])
    index = operator.index(key) # operator.index() calls the __index__ special method
    return self._components[index]

  def __getattr__(self, name):
    cls = type(self)
    try:
      pos = cls.__match_args__.index(name)
    except ValueError:
      pos = -1
    if 0 <= pos < len(self._components):
      return self._components[pos]
    msg = f'{cls.__name__!r} object has not attribute {name!r}'
    raise AttributeError(msg)

  def __setattr__(self, name, value):
    cls = type(self)
    if len(name) == 1:
      if name in cls.__match_args__:
        error = 'readonly attribute {attr_name!r}'
      elif name.islower():
        error = "can't set attribute 'a' to 'z' in {cls_name!r}"
      else:
        error = ''
      if error:
        msg = error.format(cls_name=cls.__name__, attr_name=name)
        raise AttributeError(msg)
    super().__setattr__(name, value)

  @classmethod
  def frombytes(cls, octets):
    typecode = chr(octets[0])
    memv = memoryview(octets[1:]).cast(typecode)
    return cls(memv)

In [12]:
v = Vector(range(5))

In [13]:
v

Vector([0.0, 1.0, 2.0, 3.0, 4.0])

In [14]:
v.x

0.0

In [15]:
v.x = 10

AttributeError: readonly attribute 'x'

In [16]:
v

Vector([0.0, 1.0, 2.0, 3.0, 4.0])

In [17]:
v.x

0.0

# Vector Take #4: Hashing and a Faster `==`

In [18]:
2 * 3 * 4 * 5

120

In [19]:
import functools

functools.reduce(lambda a, b: a * b, range(1, 6))

120

In [20]:
1 ^ 2

3

In [21]:
n = 0
for i in range(1, 6):
  n ^= i
  print(n)

1
3
0
4
1


In [22]:
functools.reduce(lambda a, b: a ^ b, range(6))

1

In [23]:
import operator
functools.reduce(operator.xor, range(6))

1

In [24]:
# vector_v4.py
from array import array
import operator
import reprlib
import math
import functools

class Vector:
  typecode = 'd'
  __match_args__ = ('x', 'y', 'z', 't')

  def __init__(self, components):
    # self._components - "protected" instance attribute will hold an array with the Vector components
    self._components = array(self.typecode, components)

  def __iter__(self):
    return iter(self._components)

  def __repr__(self):
    components = reprlib.repr(self._components)
    components = components[components.find('['):-1]
    return f'Vector({components})'

  def __str__(self):
    return str(tuple(self))

  def __bytes__(self):
    return (bytes([ord(self.typecode)]) +
            bytes(self._components))

  def __eq__(self, other):
    return tuple(self) == tuple(other)

  def __abs__(self):
    return math.hypot(*self)

  def __bool__(self):
    return bool(abs(self))

  def __len__(self):
    return len(self._components)

  def __getitem__(self, key):
    if isinstance(key, slice):
      cls = type(self)
      return cls(self._components[key])
    index = operator.index(key) # operator.index() calls the __index__ special method
    return self._components[index]

  def __getattr__(self, name):
    cls = type(self)
    try:
      pos = cls.__match_args__.index(name)
    except ValueError:
      pos = -1
    if 0 <= pos < len(self._components):
      return self._components[pos]
    msg = f'{cls.__name__!r} object has not attribute {name!r}'
    raise AttributeError(msg)

  def __setattr__(self, name, value):
    cls = type(self)
    if len(name) == 1:
      if name in cls.__match_args__:
        error = 'readonly attribute {attr_name!r}'
      elif name.islower():
        error = "can't set attribute 'a' to 'z' in {cls_name!r}"
      else:
        error = ''
      if error:
        msg = error.format(cls_name=cls.__name__, attr_name=name)
        raise AttributeError(msg)
    super().__setattr__(name, value)

  def __eq__(self, other):
    return len(self) == len(other) and all(a == b for a, b in zip(self, other))

  def __hash__(self):
    hashes = map(hash, self._components)
    return functools.reduce(operator.xor, hashes, 0)

  @classmethod
  def frombytes(cls, octets):
    typecode = chr(octets[0])
    memv = memoryview(octets[1:]).cast(typecode)
    return cls(memv)

In [25]:
zip(range(3), 'ABC') # zip returns a generator that produces tuples on demand

In [26]:
list(zip(range(3), 'ABC'))

[(0, 'A'), (1, 'B'), (2, 'C')]

In [29]:
list(zip(range(3), 'ABC', [0.0, 1.1, 2.2, 3.3]))

[(0, 'A', 0.0), (1, 'B', 1.1), (2, 'C', 2.2)]

In [30]:
from itertools import zip_longest

In [31]:
list(zip_longest(range(3), 'ABC', [0.0, 1.1, 2.2, 3.3]))

[(0, 'A', 0.0), (1, 'B', 1.1), (2, 'C', 2.2), (None, None, 3.3)]

In [32]:
a = [(1, 2, 3),
     (4, 5, 6)]
list(zip(*a))

[(1, 4), (2, 5), (3, 6)]

In [33]:
b = [(1, 2),
     (3, 4),
     (5, 6)]
list(zip(*b))

[(1, 3, 5), (2, 4, 6)]

# Vector Take #5: Formatting

In [36]:
# vector_v5.py
from array import array
import operator
import reprlib
import math
import functools
import itertools

class Vector:
  typecode = 'd'
  __match_args__ = ('x', 'y', 'z', 't')

  def __init__(self, components):
    # self._components - "protected" instance attribute will hold an array with the Vector components
    self._components = array(self.typecode, components)

  def __iter__(self):
    return iter(self._components)

  def __repr__(self):
    components = reprlib.repr(self._components)
    components = components[components.find('['):-1]
    return f'Vector({components})'

  def __str__(self):
    return str(tuple(self))

  def __bytes__(self):
    return (bytes([ord(self.typecode)]) +
            bytes(self._components))

  def __eq__(self, other):
    return tuple(self) == tuple(other)

  def __abs__(self):
    return math.hypot(*self)

  def __bool__(self):
    return bool(abs(self))

  def __len__(self):
    return len(self._components)

  def __getitem__(self, key):
    if isinstance(key, slice):
      cls = type(self)
      return cls(self._components[key])
    index = operator.index(key) # operator.index() calls the __index__ special method
    return self._components[index]

  def __getattr__(self, name):
    cls = type(self)
    try:
      pos = cls.__match_args__.index(name)
    except ValueError:
      pos = -1
    if 0 <= pos < len(self._components):
      return self._components[pos]
    msg = f'{cls.__name__!r} object has not attribute {name!r}'
    raise AttributeError(msg)

  def __setattr__(self, name, value):
    cls = type(self)
    if len(name) == 1:
      if name in cls.__match_args__:
        error = 'readonly attribute {attr_name!r}'
      elif name.islower():
        error = "can't set attribute 'a' to 'z' in {cls_name!r}"
      else:
        error = ''
      if error:
        msg = error.format(cls_name=cls.__name__, attr_name=name)
        raise AttributeError(msg)
    super().__setattr__(name, value)

  def __eq__(self, other):
    return len(self) == len(other) and all(a == b for a, b in zip(self, other))

  def __hash__(self):
    hashes = map(hash, self._components)
    return functools.reduce(operator.xor, hashes, 0)

  def angle(self, n):
    r = math.hypot(*self[n:])
    a = math.atan2(r, self[n-1])
    if (n == len(self) - 1) and (self[-1] < 0):
      return math.pi * 2 - a
    else:
      return a

  def angles(self):
    return(self.angle(n) for n in range(1, len(self)))

  def __format__(self, fmt_spec=''):
    if fmt_spec.endswith('h'):
      fmt_spec = fmt_spec[:-1]
      coords = itertools.chain([abs(self)], self.angles())
      outer_fmt = '<{}>'
    else:
      coords = self
      outer_fmt = '({})'
    components = (format(c, fmt_spec) for c in coords)
    return outer_fmt.format(', '.join(components))

  @classmethod
  def frombytes(cls, octets):
    typecode = chr(octets[0])
    memv = memoryview(octets[1:]).cast(typecode)
    return cls(memv)

In [39]:
format(Vector([-1, -1, -1, -1]), 'h')
format(Vector([2, 2, 2, 2]), '.3eh')
format(Vector([0, 1, 0, 0]), '.5fh')

'<1.00000, 1.57080, 0.00000, 0.00000>'